In [1]:
import numpy as np

def gaussQuadStd1d(g, noOfIntegPt):
    """Computes the integral ∫ g(ξ) dξ using Gaussian quadrature on [-1,1]"""
    if noOfIntegPt == 2:
        points = np.array([-1/np.sqrt(3), 1/np.sqrt(3)])
        weights = np.array([1, 1])
    elif noOfIntegPt == 3:
        points = np.array([-np.sqrt(3/5), 0, np.sqrt(3/5)])
        weights = np.array([5/9, 8/9, 5/9])
    else:
        return 0
    
    return sum(w * g(xi) for w, xi in zip(weights, points))

def gaussQuad1d(fn, lowerLimit, upperLimit, noOfIntegPt):
    transform = lambda ξ: (upperLimit - lowerLimit) / 2 * ξ + (upperLimit + lowerLimit) / 2
    g = lambda ξ: fn(transform(ξ)) * (upperLimit - lowerLimit) / 2 

    return gaussQuadStd1d(g, noOfIntegPt)

In [2]:
from scipy.optimize import root_scalar

def midPtCurve(f, fder, x1, x2):
    """
    Find the midpoint along the curve y = f(x) from x1 to x2 in arc length.

    inputs:
    - f: function f(x)
    - fder: derivative f'(x)
    - x1, x2: x-coordinates of curve segment endpoints

    output:
    - (xm, ym): coordinates of the midpoint along the curve
    """

    # Define the arc length integrand
    #def arc_length_integrand(x):
     #   return np.sqrt(1 + (fder(x))**2)
    arc_length_integrand = lambda t: np.sqrt(1 + (fder(t))**2)

    # Compute total arc length (L) from x1 to x2
    L = gaussQuad1d(arc_length_integrand, x1, x2, 2)
    
    # Define the arc length function (l(x)) from x1 to x
    l = lambda x: gaussQuad1d(arc_length_integrand, x1, x, 2)

    # Define m(x) = l(x) - L/2
    m = lambda x: l(x) - L/2

    # Use root-finding to solve m(xm) = 0
    sol = root_scalar(m, bracket=[x1, x2], method='brentq', xtol=1e-6)

    if not sol.converged:
        raise ValueError("Root finding for midpoint did not converge.")

    xm = sol.root
    ym = f(xm)

    return xm, ym

In [3]:
import numpy as np

f1 = lambda x: x**3 - 5*x
f1der = lambda x: 3*x**2 - 5
x1 = -12.4
x2 = -8.12
xm, ym = midPtCurve(f1,f1der,x1,x2)
print(f'mid point = ({xm},{ym})')

mid point = (-10.694564612658947,-1169.704232630375)


In [4]:
f2 = lambda x: np.cos(x) - 0.22*x**3
f2der = lambda x: -np.sin(x) - 0.66*x**2
x1 = -1
x2 = 0.5
xm, ym = midPtCurve(f2,f2der,x1,x2)
print(f'mid point = ({xm},{ym})')

mid point = (-0.25251771525908207,0.97182886391468)


In [5]:
from scipy.spatial import Delaunay

def delaunayMesh(nodes):
    """
    Generate a mesh for a QUADRILATERAL domain given by specified nodes
    {(x, y) | |x|+2|y| ≤ 4}
    
    inputs: 
     - nodes: 2D numpy array with 2 columns representing the x- and y- coordinates of the nodes.
     
    outputs:
     - triangles: 2D numpy array with 3 columns, the e-th row lists the 3 vertices of the e-th element.
     - edges: 2D numpy array with 2 columns. In each row, two vertices of one edge is listed.
     - bdyNode: numpy boolean array, the i-th element determines the i-th node's nature as a boundary
     - bdyEdge: numpy boolean array, the i-th element determines the i-th node's nature as a boundary
    """
    # Verify nodes is of acceptable dimension and nonzero
    if not nodes.ndim == 2 and nodes.shape[0] > 2:
        raise ValueError("nodes is of unexpected dimensions")
    
    # Initialize bdyNode array
    bdyNode = np.zeros(len(nodes), dtype = bool)
    
    # loop through nodes array to determine the nature of each node
    for i in range(len(nodes)):
        if np.abs(nodes[i,0]) + 2*np.abs(nodes[i,1]) == 4:
            bdyNode[i] = True
        else:
            bdyNode[i] = False
    
    # Set up triangles matrix, accounting for the first node being 1 NOT 0
    triangles = Delaunay(nodes).simplices + np.ones_like(Delaunay(nodes).simplices)
    
    # Initialize edges matrix, 3 edges for each triangle in the mesh
    edges = np.zeros((3*len(triangles), 2))
    for i in range(len(triangles)):
        edges[3*i, :] = np.array([triangles[i,0], triangles[i,1]])
        edges[3*i + 1, :] = np.array([triangles[i,0], triangles[i,2]])
        edges[3*i + 2, :] = np.array([triangles[i,1], triangles[i,2]])
        
    edges = np.sort(edges, axis=1) # sort matrix by column to make filtering repeated edges easier
    edges, counts = np.unique(edges, axis=0, return_counts = True)
    edges = edges[1:]
    counts = counts[1:]
    
    # for boundary edges make sure it is only present in one triangle, i.e only on one row of triangles matrix
    bdyEdge = np.zeros(len(edges), dtype = bool) # size is of len(edges)x1
    for i in range(len(edges)):
        if counts[i] > 1:
            bdyEdge[i] = False
        else:
            bdyEdge[i] = True
    
    return triangles, edges, bdyNode, bdyEdge

In [6]:
nodes1 = np.array([[-1,0],[-0.5,-0.5],[0,-1],[0.5,-0.5],[1,0],[0.5,0.5],[0,1],[-0.5,0.5],[0,0]])
triangles1,edges1, bdyNode1, bdyEdge1 = delaunayMesh(nodes1)
print(f'triangle = {triangles1}')
print(f'edges = {edges1}')
print(f'bdyEdges = {bdyEdge1}')

triangle = [[2 4 9]
 [4 2 3]
 [8 2 9]
 [2 8 1]
 [4 6 9]
 [6 4 5]
 [6 8 9]
 [8 6 7]]
edges = [[1. 8.]
 [2. 3.]
 [2. 4.]
 [2. 8.]
 [2. 9.]
 [3. 4.]
 [4. 5.]
 [4. 6.]
 [4. 9.]
 [5. 6.]
 [6. 7.]
 [6. 8.]
 [6. 9.]
 [7. 8.]
 [8. 9.]]
bdyEdges = [ True  True False False False  True  True False False  True  True False
 False  True False]
